![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

Maintenant nous avons deux résultats :  
- Un fichier csv contenant la géolocalisation et la météo des villes
- Un fichier json contenant les hotels, notes... pour les villes sélectionées.  
  
L'objectif est double :  
- Créer un dataframe avec les valerus du fichier json
- Enregistrer nos données concernant les villes et les hôtels dans un dataLake (AWS)

In [1]:
import pandas as pd
import plotly.express as px
import boto3

In [2]:
df = pd.read_csv('destinations.csv')
df.head()

,id_ville,Ville,latitude,longitude,main_weather,expected_rain
0,7,Lille,50.636565,3.063528,Clouds,3.6870
1,5,Paris,48.856697,2.351462,Clear,3.7847
2,6,Amiens,49.894171,2.295695,Clouds,4.5048
3,0,Mont Saint Michel,48.635523,-1.510257,Rain,5.3364
4,4,Rouen,49.440459,1.093966,Clouds,6.4300


In [4]:
ville = input("Quelle ville aviez vous choisi ?")

Quelle ville aviez vous choisi ? Paris


In [5]:
hotels = pd.DataFrame(columns = ['city_id', 'city_name', 'hotel_id']) #Création d'un dataframe


temp_dataset = pd.read_json('hotel_{}.json'.format(ville)) # On récupère le document json de la ville étudiée
temp_dataset = temp_dataset.reset_index().rename({'index': 'hotel_id'}, axis = 1) # Changement de la colonne index
temp_dataset.loc[:,'city_id'] = df['id_ville'][0] # On récupère l'id de la ville du df, pour l'associer à city_id
temp_dataset.loc[:,'city_name'] = ville # Dans la colonne "city_name" on ajoute le nom de la ville choisi 
    
# préprocessing des valeurs scrappées
temp_dataset.loc[:, 'hotel_name'] = temp_dataset['hotel_name'].str.replace('\n', '') 
temp_dataset.loc[:, 'hotel_url'] = temp_dataset['hotel_url'].str.replace('\n', '')
temp_dataset.loc[:, 'hotel_description'] = temp_dataset['hotel_description'].str.replace('\n', '')


hotels = hotels.append(temp_dataset) # On ajoute l'ensemble des valeurs dans le dataframe "hotels"

In [6]:
hotels.head()

,city_id,city_name,hotel_id,hotel_name,hotel_url,hotel_note,hotel_description
0,7,Paris,0,ibis budget Paris Porte de Vincennes,/hotel/fr/ibis-budget-paris-porte-de-vincennes...,"7,1","Situé à Paris, l'ibis budget Paris Porte de Vi..."
1,7,Paris,1,hotelF1 Paris Porte de Châtillon,/hotel/fr/hotelf1-paris-porte-de-chatillon-par...,"7,4","Situé dans le 14ème arrondissement de Paris, à..."
2,7,Paris,2,Kyriad Paris 10 - Canal Saint Martin - République,/hotel/fr/kyriad-paris-10-canal-saint-martin-r...,"7,5",Cet hôtel Kyriad se situe tout près de la plac...
3,7,Paris,3,ibis Budget Paris La Villette 19ème,/hotel/fr/ibis-budget-paris-la-villette.fr.htm...,"7,4",L'ibis Budget Paris La Villette 19ème est situ...
4,7,Paris,4,Hôtel Jarry Confort,/hotel/fr/jarry-confort.fr.html?label=gen173nr...,"6,4",L'Hôtel Jarry Confort se trouve dans le 10ème ...


In [17]:
hotels.to_csv('hotels.csv', index=False) # enregistrment en csv

In [ ]:
# Crétation dans Data Lake dans une bucket S3 de AWS

s3 = boto3.resource("s3")  # s3 = session.resource("s3")
bucket_name = s3.create_bucket(Bucket="jedha_kayak") 


hotels_csv = hotels.to_csv(index=False)
put_object = bucket_name.put_object(Key = "hotels.csv", Body = hotels_csv)


cities_csv = df.to_csv(index=False)
put_object = bucket_name.put_object(Key = "cities.csv", Body = cities_csv)